In [1]:
import tensorflow as tf
from IPython.display import display
import pandas as pd
import seaborn as sns
import tensorflow.keras.backend as K
import numpy as np
import os
import gc
import glob
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import BatchNormalization, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
%matplotlib inline

In [2]:
def tf_reset_graph(model=None):
    tf.reset_default_graph()
    K.clear_session()
    gc.collect()

def tf_reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint = None
    reduce_lr = None
    early_stopping = None
    tensorboard = None

In [3]:
use_colab = False
if use_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  proj_path = 'drive/My Drive/Colab/Workspace/'
  import platform
  print("Processor: ", platform.processor())
  !nvidia-smi
else:
  proj_path=''

In [4]:
image_ext = '.jpeg'
data_dir = proj_path+'dataset/'
test_dir = data_dir+'test/'
train_dir = data_dir+'train/'
val_dir = data_dir+'val/'
normals = 'NORMAL/'
infected = 'PNEUMONIA/'
training_normal_nos = len(glob.glob1(train_dir+normals,'*'+image_ext))
training_infected_nos = len(glob.glob1(train_dir+infected,'*'+image_ext))
training_nos = training_infected_nos+training_normal_nos
testing_normal_nos = len(glob.glob1(test_dir+normals,'*'+image_ext))
testing_infected_nos = len(glob.glob1(test_dir+infected,'*'+image_ext))
testing_nos = testing_infected_nos+testing_normal_nos
validation_normal_nos = len(glob.glob1(val_dir+normals,'*'+image_ext))
validation_infected_nos = len(glob.glob1(val_dir+infected,'*'+image_ext))
validation_nos = validation_infected_nos+validation_normal_nos

print('Total number of training images: ', training_nos)
print('Total number of testing images: ', testing_nos)
print('Total number of validation images: ', validation_nos)

Total number of training images:  5216
Total number of testing images:  624
Total number of validation images:  16


In [5]:
def create_image_data_generator(target_size, rescale, path, batch_size, class_mode = 'categorical', shuffle=True, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True):

    data_generator = ImageDataGenerator(rescale=rescale,
                                        shear_range=shear_range,
                                        zoom_range=zoom_range,
                                        horizontal_flip=horizontal_flip)

    return data_generator.flow_from_directory(path,
                                       class_mode=class_mode,
                                       target_size=target_size,
                                       shuffle=shuffle,
                                       batch_size=batch_size)

In [6]:
rescale_factor = 1./255
target_size = (150,150)
class_mode = 'categorical'
training_batch_size = 163
train_data_generator = create_image_data_generator(target_size=target_size, 
                                                   rescale=rescale_factor, 
                                                   path=train_dir, 
                                                   batch_size=training_batch_size)

test_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=test_dir,
                                                 batch_size=testing_nos)

val_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=val_dir,
                                                 batch_size=validation_nos)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [7]:
tf_reset_callbacks()
tf_reset_graph()
base = proj_path+'bin'
model_dir = os.path.join(base,'models')
log_dir = os.path.join(base,'logs')
model_file = model_dir + "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"

checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_acc', 
    save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True)

tensorboard = TensorBoard(
    log_dir=log_dir,
    batch_size=training_batch_size,
    update_freq = 'batch')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=5,
    cooldown=2,
    min_lr=1e-10,
    verbose=1)

#callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
callbacks = [checkpoint, reduce_lr, tensorboard]

In [10]:
def create_model():
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape = target_size+(3,))
    x = inception_model.output
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.15)(x)
    x = Dense(256, activation='relu')(x)
    x = Flatten()(x)
    predictions = Dense(2,activation='softmax')(x)
    model = Model(inputs = inception_model.input, outputs=predictions)
    for layer in inception_model.layers:
        layer.trainable = False
    return model

In [11]:
steps_per_epoch = len(train_data_generator)
validation_steps = len(val_data_generator)
classes = train_data_generator.classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(classes), classes)

optimizer = SGD()#Adam()
model = create_model()
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_data_generator,
                             steps_per_epoch=steps_per_epoch,
                             epochs=100,
                             verbose=1,
                             validation_data=val_data_generator,
                             validation_steps=validation_steps,
                             class_weight=class_weights,
                             callbacks=callbacks)

Epoch 1/100


In [0]:
model_file = 'pneumonia_trained.hd5'
model.save(model_file)